# Step-by-step run of alphazero self-play & training.


In [1]:
import os
import time
from pathlib import Path
from collections import defaultdict, Counter
import asyncio
from typing import Callable

import numpy as np
import torch
import torch.nn.functional as F

# Game and players
from rgi.rgizero.games.connect4 import Connect4Game
from rgi.rgizero.players.alphazero import AlphazeroPlayer
from rgi.rgizero.players.alphazero import play_game

from notebook_utils import reload_local_modules

print("✅ Imports successful")

if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available():
  device = 'mps'
else:
    device = 'cpu'
print(f'Using device: {device}')
assert device in ('cuda', 'mps'), f"No accelerator available, device={device}"

# Allow asyncio to work with jupyter notebook
import nest_asyncio
nest_asyncio.apply()

# Increase numpy print width
np.set_printoptions(linewidth=300)

%load_ext line_profiler

✅ Imports successful
Using device: mps


In [2]:
DEBUG_MODE = True     # Set options to make debugger work properly. Single worker, etc.
LOAD_MODEL = False
TRAIN_MODEL = True
MODEL_SIZE = "tiny"  # "tiny" or "small" or "large" or "xl"
NUM_SIMULATIONS = 20
RUN_GENERATIONS = False

# If False, we still load previous games from disk.
NUM_GAMES = 10_000
MAX_TRAINING_ITERS = 1_000_000
CONFIG_ALIAS = f'trajectory_sims-{NUM_SIMULATIONS}_games-{NUM_GAMES}_size-{MODEL_SIZE}_train-{MAX_TRAINING_ITERS}_x1'
NUM_GENERATIONS = 1

# DEBUG: Update batch_size after config_alias
MAX_TRAINING_ITERS = 10_000_000
MAX_TRAINING_EPOCHS = 1_000_000


## Step 1: Set up history-wrapped game


In [3]:
from rgi.rgizero.games.history_wrapper import HistoryTrackingGame
from rgi.rgizero.data.trajectory_dataset import Vocab
from rgi.rgizero.common import TOKENS

base_game, max_game_length = Connect4Game(connect_length=4), 7*6

game = HistoryTrackingGame(base_game)
state_0 = game.initial_state()
block_size = max_game_length + 2
all_actions = game.all_actions()
action_vocab = Vocab(itos=[TOKENS.START_OF_GAME] + list(all_actions))
n_max_context = max_game_length + 2
game_name = base_game.__class__.__name__

print("✅ Using HistoryTrackingGame from module")
print(f"Game: {game_name}, Players: {game.num_players(state_0)}, Actions: {list(game.all_actions())}")

DATA_DIR = Path.cwd().parent / "data" / "rgizero-e2e" / game_name / CONFIG_ALIAS
print("Creating data dir: ", DATA_DIR)
os.makedirs(DATA_DIR, exist_ok=True)

MODEL_DIR = Path.cwd().parent / "models" / "rgizero-e2e" / game_name / CONFIG_ALIAS
print("Creating model dir: ", MODEL_DIR)
os.makedirs(MODEL_DIR, exist_ok=True)


✅ Using HistoryTrackingGame from module
Game: Connect4Game, Players: 2, Actions: [1, 2, 3, 4, 5, 6, 7]
Creating data dir:  /Users/rodo/src/rgi3/data/rgizero-e2e/Connect4Game/trajectory_sims-20_games-10000_size-tiny_train-1000000_x1
Creating model dir:  /Users/rodo/src/rgi3/models/rgizero-e2e/Connect4Game/trajectory_sims-20_games-10000_size-tiny_train-1000000_x1


## Step 2: Create random generation_0 model


In [4]:
def print_dataset_stats(dataset_path, split_name):
    """Print statistics about a loaded trajectory dataset."""
    td = TrajectoryDataset(dataset_path.parent, split_name, block_size=n_max_context)
    
    # Calculate basic stats
    num_trajectories = len(td)
    total_actions = td._num_actions
    avg_trajectory_length = total_actions / num_trajectories if num_trajectories > 0 else 0
    
    # Get trajectory lengths, winners, and first moves
    trajectory_lengths = []
    winners = []
    first_moves = []
    
    for i in range(num_trajectories):
        start_idx = td.boundaries[i]
        end_idx = td.boundaries[i + 1]
        traj_length = end_idx - start_idx
        trajectory_lengths.append(traj_length)
        
        # Get winner from final values (values are the same throughout trajectory)
        # Values are in range [-1, 1] where positive means player 1 advantage
        final_values = td.value_data[start_idx]  # shape: (num_players,)
        if final_values[0] > final_values[1]:
            winners.append(1)
        elif final_values[1] > final_values[0]:
            winners.append(2)
        else:
            winners.append(None)  # Draw
        
        # Get first move (decode from vocab)
        first_action_encoded = td.action_data[start_idx]
        first_action = action_vocab.decode([first_action_encoded])[0]
        first_moves.append(first_action)
    
    # Print basic stats
    print(f"Dataset Stats:")
    print(f"  Trajectories: {num_trajectories}")
    print(f"  Total actions: {total_actions}")
    print(f"  Avg trajectory length: {avg_trajectory_length:.2f}")
    print(f"  Trajectory length - min: {min(trajectory_lengths)}, max: {max(trajectory_lengths)}, mean: {np.mean(trajectory_lengths):.2f}")
    
    # Print winner stats (similar to print_game_stats)
    print(f"Winner Stats:")
    winner_stats = Counter(winners)
    total_games = num_trajectories
    win1_pct = 100 * winner_stats[1] / total_games if total_games > 0 else 0
    win2_pct = 100 * winner_stats[2] / total_games if total_games > 0 else 0
    print(f"  Winner counts: win[1]={win1_pct:.2f}% win[2]={win2_pct:.2f}%, n={total_games}")
    
    # Print stats by initial move
    print(f"Winner Stats by initial move:")
    move_stats = defaultdict(Counter)
    for first_move, winner in zip(first_moves, winners):
        move_stats[first_move][winner] += 1
    
    for action in sorted(move_stats.keys()):
        counts = move_stats[action]
        total = sum(counts.values())
        win1_pct = 100 * counts[1] / total if total > 0 else 0
        print(f"  a={action}: n={total:3} win[1]={win1_pct:.2f}% counts={counts}")

def print_model_stats(model, config_alias=""):
    """Print statistics about a model."""
    # Count parameters
    num_params = sum(p.numel() for p in model.parameters())
    num_trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    print(f"Model Stats:")
    print(f"  Config: {model.config}")
    print(f"  Total parameters: {num_params:,}")
    print(f"  Trainable parameters: {num_trainable:,}")
    if config_alias:
        print(f"  Config alias: {config_alias}")


In [5]:
reload_local_modules(verbose=False)

from rgi.rgizero.models.action_history_transformer import ActionHistoryTransformer
from rgi.rgizero.models.transformer import TransformerConfig

model_config_dict = {
    "tiny": TransformerConfig(n_max_context=n_max_context, n_layer=2, n_head=2, n_embd=8),
    "small": TransformerConfig(n_max_context=n_max_context, n_layer=4, n_head=4, n_embd=32),
    "large": TransformerConfig(n_max_context=n_max_context, n_layer=8, n_head=8, n_embd=128),
    "xl": TransformerConfig(n_max_context=n_max_context, n_layer=16, n_head=16, n_embd=256),
}


def create_random_model(config: TransformerConfig, action_vocab_size, num_players,  seed: int):
    torch.manual_seed(seed)
    np.random.seed(seed) # Ensure numpy operations are also seeded
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    model = ActionHistoryTransformer(config=config, action_vocab_size=action_vocab_size, num_players=num_players)
    model.to(device)
    return model

# Make model initialization deterministic
model_config = model_config_dict[MODEL_SIZE]
model_0 = create_random_model(model_config, action_vocab_size=action_vocab.vocab_size, num_players=game.num_players(state_0), seed=42)

# Step 3: Define play & generation code


In [6]:
from rgi.rgizero.models.action_history_transformer import AsyncNetworkEvaluator, ActionHistoryTransformerEvaluator
from rgi.rgizero.players.alphazero import play_game_async
from tqdm.asyncio import tqdm

async def play_games_async(num_games: int, player_factory: Callable[[], AlphazeroPlayer]):
    tasks = []
    async def create_player_and_create_game():
        t0 = time.time()
        player = player_factory()
        game_result = await play_game_async(game, [player, player])
        t1 = time.time()
        game_result['time'] = t1 - t0
        return game_result

    tasks = [create_player_and_create_game() for _ in range(num_games)]
    results = await tqdm.gather(*tasks)   # same as asyncio.gather, but with a progress bar
    return results

async def play_generation_async(model, num_games, simulations=NUM_SIMULATIONS):
    serial_evaluator = ActionHistoryTransformerEvaluator(model, device=device, block_size=block_size, vocab=action_vocab)
    async_evaluator = AsyncNetworkEvaluator(base_evaluator=serial_evaluator, max_batch_size=1024, verbose=False)

    master_rng = np.random.default_rng(42)
    async_player_factory = lambda: AlphazeroPlayer(game, async_evaluator, rng=np.random.default_rng(master_rng.integers(0, 2**31)), add_noise=False, simulations=simulations)

    await async_evaluator.start()
    results = await play_games_async(num_games=num_games, player_factory=async_player_factory)
    await async_evaluator.stop()
    return results

def print_game_stats(results):
    print("Winner Stats:")
    winner_stats = Counter(result['winner'] for result in results)
    print(f"Winner counts: win[1]={100*winner_stats[1]/sum(winner_stats.values()):.2f}% win[2]={100*winner_stats[2]/sum(winner_stats.values()):.2f}%, n={sum(winner_stats.values())}")
    game_lengths = [len(result['action_history']) for result in results]
    print(f"Game Length min: {min(game_lengths)}, max: {max(game_lengths)}, mean: {np.mean(game_lengths):.2f}")
    print("Winner Stats by initial move:")
    dd = defaultdict(Counter)
    for result in results:
        dd[result['action_history'][0]][result['winner']] += 1
    for action, counts in sorted(dd.items()):
        print(f"  a={action}: n={sum(counts.values()):3} win[1]={100*counts[1]/sum(counts.values()):.2f}% counts={counts}")

## Step 3: Confirm we can read & write to trajectory_dataset


In [7]:
from rgi.rgizero.data.trajectory_dataset import TrajectoryDatasetBuilder, TrajectoryDataset, build_trajectory_loader
reload_local_modules(verbose=False)

def add_trajectory(game_result, vocab, td_builder):
    action_history = game_result['action_history']
    trajectory_length = len(action_history)
    legal_policies = game_result['legal_policies']
    legal_action_idx = game_result['legal_action_idx']
    rewards = game_result['rewards']

    # Translation key for converting legal_action_ids to vocab_action_idx.
    action_idx_to_vocab_idx = vocab.encode(all_actions)

    fixed_width_policies = np.zeros((trajectory_length, vocab.vocab_size))
    for i in range(trajectory_length):
        vocab_action_idx = action_idx_to_vocab_idx[legal_action_idx[i]]
        fixed_width_policies[i, vocab_action_idx] = legal_policies[i]

    encoded_action_history = vocab.encode(action_history)
    tiled_rewards = np.tile(rewards, (trajectory_length, 1))  # shape (num_players,) -> (num_moves, num_players)
    
    td_builder.add_trajectory(actions=encoded_action_history, fixed_width_policies=fixed_width_policies, values=tiled_rewards)

def write_trajectory_dataset(results, action_vocab, generation_id):
    td_builder = TrajectoryDatasetBuilder(action_vocab)
    for game_result in results:
        add_trajectory(game_result, action_vocab, td_builder)

    trajectory_path = td_builder.save(DATA_DIR, f"gen-{generation_id}")
    return trajectory_path

## Train model


In [8]:
from rgi.rgizero.train import Trainer, TrainConfig

train_config = TrainConfig(
    model_name="connect4-e2e",
    model_version="v1",

    eval_interval = 1000,  # keep frequent because we'll overfit
    eval_iters = 20,
    log_interval = 100,  # don't print too too often
    max_epochs = MAX_TRAINING_EPOCHS,

    # we expect to overfit on this small dataset, so only save when val improves
    always_save_checkpoint = False,

    gradient_accumulation_steps = 1,
    batch_size = 2048,

    learning_rate = 1e-3,  # with baby networks can afford to go a bit higher
    max_iters = MAX_TRAINING_ITERS,
    lr_decay_iters = 5000,  # make equal to max_iters usually
    min_lr = 1e-4,  # learning_rate / 10 usually
    beta2 = 0.99,  # make a bit bigger because number of tokens per iter is small

    warmup_iters = 100,  # not super necessary potentially
)

def train_model(model, training_splits, train_config):
    # Load dataset
    num_workers = 0 if DEBUG_MODE else 4

    trajectory_loader = build_trajectory_loader(
        DATA_DIR, training_splits, block_size=n_max_context, batch_size=train_config.batch_size,
        device=device, workers=num_workers, shuffle=True)
        
    trainer = Trainer(
        model=model,
        train_config=train_config,
        train_loader=trajectory_loader,
        val_loader=trajectory_loader,  # TODO: Create separate validation loader
        device=device
    )

    trainer.train()
    return model, trainer

In [9]:
import dataclasses

def get_model_path(generation_id):
    return MODEL_DIR / f"gen-{generation_id}.pt"

def save_model(model, trainer, generation_id):
    # Save model
    model_path = get_model_path(generation_id)

    checkpoint = {
        'model': model.state_dict(),
        'model_config': dataclasses.asdict(model.config),
        'vocab': action_vocab.to_dict(),
        'iter_num': trainer.iter_num,
        'best_val_loss': trainer.best_val_loss,
        'num_players': game.num_players(state_0),
    }
    torch.save(checkpoint, model_path)
    return model_path

def load_model(generation_id):
    model_path = get_model_path(generation_id)
    loaded_checkpoint = torch.load(model_path)
    loaded_model = ActionHistoryTransformer(
        config=TransformerConfig(**loaded_checkpoint['model_config']),
        action_vocab_size=loaded_checkpoint['vocab']['vocab_size'],
        num_players=loaded_checkpoint['num_players']
    )
    loaded_model.load_state_dict(loaded_checkpoint['model']) 
    loaded_model.to(device)
    return loaded_model


In [10]:
# Do a single generation of play & train
async def run_generation(model, num_games, num_simulations, generation_id):
    print(f"\n\n## Running generation {generation_id} for config_alias={CONFIG_ALIAS}")
    split_name = f"gen-{generation_id}"
    expected_trajectory_path = DATA_DIR / split_name
    if not expected_trajectory_path.exists():
        print(f"Playing {num_games} games, simulations={num_simulations}, model_size={MODEL_SIZE}")
        results = await play_generation_async(model, num_games=NUM_GAMES, simulations=NUM_SIMULATIONS)
        print_game_stats(results)
        trajectory_path = write_trajectory_dataset(results, action_vocab, generation_id)
        assert trajectory_path == expected_trajectory_path
    else:
        print(f"Loading trajectory from {expected_trajectory_path}")
        print_dataset_stats(expected_trajectory_path, split_name)
        trajectory_path = expected_trajectory_path
        results = None

    model_path = get_model_path(generation_id)
    if not model_path.exists():
        print(f"Training model on {split_name}")
        training_splits = [f"gen-{i}" for i in range(1, generation_id+1)]
        # TODO: We're continuing training on a previosu model here ... should we train a new model from scratch?
        updated_model, trainer = train_model(model, training_splits, train_config)
        save_model(updated_model, trainer, generation_id)
    else:
        print(f"Loading model from {model_path}")
        updated_model = load_model(generation_id)
        print_model_stats(updated_model, config_alias=MODEL_SIZE)

    return results, trajectory_path, updated_model

In [ ]:
results_dict = {}
trajectory_paths_dict = {}
model_dict = {0: model_0}

# 10 hours!
if RUN_GENERATIONS:
    for generation_id in range(1, NUM_GENERATIONS+1):
        current_model = model_dict[generation_id-1]
        results_i, trajectory_path_i, model_i = await run_generation(current_model, num_games=NUM_GAMES, num_simulations=NUM_SIMULATIONS, generation_id=generation_id)
        results_dict[generation_id] = results_i
        trajectory_paths_dict[generation_id] = trajectory_path_i
        model_dict[generation_id] = model_i




## Running generation 1 for config_alias=trajectory_sims-20_games-10000_size-tiny_train-1000000_x1
Loading trajectory from /Users/rodo/src/rgi3/data/rgizero-e2e/Connect4Game/trajectory_sims-20_games-10000_size-tiny_train-1000000_x1/gen-1
Dataset Stats:
  Trajectories: 10000
  Total actions: 155412
  Avg trajectory length: 15.54
  Trajectory length - min: 7, max: 42, mean: 15.54
Winner Stats:
  Winner counts: win[1]=56.78% win[2]=43.22%, n=10000
Winner Stats by initial move:
  a=1: n=2085 win[1]=52.81% counts=Counter({1: 1101, 2: 984})
  a=2: n=989 win[1]=51.26% counts=Counter({1: 507, 2: 482})
  a=3: n=999 win[1]=62.16% counts=Counter({1: 621, 2: 378})
  a=4: n=1487 win[1]=63.55% counts=Counter({1: 945, 2: 542})
  a=5: n=2027 win[1]=60.98% counts=Counter({1: 1236, 2: 791})
  a=6: n=981 win[1]=55.66% counts=Counter({1: 546, 2: 435})
  a=7: n=1432 win[1]=50.42% counts=Counter({1: 722, 2: 710})
Training model on gen-1
num decayed parameter tensors: 11, with 1,968 parameters
num non-deca

# Debug


In [ ]:
# Play single game
result = await play_generation_async(current_model, num_games=1, simulations=NUM_SIMULATIONS)

In [ ]:
# Inspect training data
td_array = [TrajectoryDataset(DATA_DIR, f"gen-{generation_id}", block_size=n_max_context) for generation_id in range(1, NUM_GENERATIONS+1)]

In [ ]:
# [td for td in td_array]
unrolled = [(generation+1, d) for generation, td in enumerate(td_array) for d in td]

# gen, d = unrolled[0], 
# d.action[:2]
# d.value[0]

dd = defaultdict(lambda: defaultdict(lambda: torch.tensor([0., 0.])))

for gen, d in unrolled:
    for g in ['*', gen]:    
        dd[tuple(tuple(d.action[:0].tolist()))][g] += d.value[0]
        dd[tuple(tuple(d.action[:1].tolist()))][g] += d.value[0]
        dd[tuple(tuple(d.action[:2].tolist()))][g] += d.value[0]
        dd[tuple(tuple(d.action[:3].tolist()))][g] += d.value[0]
        dd[tuple(tuple(d.action[:4].tolist()))][g] += d.value[0]

print(f"len(dd) = {len(dd)}")


In [ ]:
def eval_prefix(model, game, prefix):
    serial_evaluator = ActionHistoryTransformerEvaluator(model, device=device, block_size=block_size, vocab=action_vocab)
    state = game.initial_state()
    for action in prefix:
        state = game.next_state(state, action)
    legal_actions = game.legal_actions(state)
    result = serial_evaluator.evaluate(game, state, legal_actions)
    return result


In [ ]:
## Someting is borked? Player1 win percent should be much higher??
def compare_model_vs_data(model, game, dd):
    prefix_list = [(), (4,), (4,1), (4,2), (4,3), (4,4), (4,5), (4,6), (4,7)]
    for prefix in prefix_list:
        print(f"\nprefix={prefix}")
        for gen, counts in dd[prefix].items():
            print(f"gen={gen}: {counts}, win_pct={100*counts[0]/sum(counts):.2f}%, sum={sum(counts)}")
        eval_result = eval_prefix(model, game, prefix)
        print(f'legal_policy={eval_result.legal_policy}')
        print(f'player_values={eval_result.player_values}')
        print(f'player_probs={(eval_result.player_values+1)/2}')

compare_model_vs_data(current_model, game, dd)


In [ ]:
# Copy model
model_0 = create_random_model(model_config, action_vocab_size=action_vocab.vocab_size, num_players=game.num_players(state_0), seed=42)
model_1 = load_model(1)

In [ ]:
print("\n\n### Model 0")
print(model_0.action_embedding.weight)
compare_model_vs_data(model_0, game, dd)

In [ ]:
print("\n\n### Model 1")
print(model_1.action_embedding.weight)
compare_model_vs_data(model_1, game, dd)

## Debug Convergence

Synthetic sanity-check: train on a toy 2-step game where the first action strongly determines the winner. This verifies the value head and training loop can learn simple patterns.


In [2]:
state_0 = game.initial_state()

NameError: name 'game' is not defined

In [1]:
# Synthetic 2-step dataset and minimal value-only training
import math
from dataclasses import dataclass
from typing import Iterable

import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

# 1) Define a synthetic dataset: trajectories of exactly length 2
# First action a1 ∈ {1..7} drives win prob for Player 1; second action is noise
GOOD_ACTIONS = {4, 3}
BAD_ACTIONS = {1, 7}
NEUTRAL_ACTIONS = {2, 5, 6}
P_WIN = {  # P1 win prob by first action
    **{a: 0.9 for a in GOOD_ACTIONS},
    **{a: 0.1 for a in BAD_ACTIONS},
    **{a: 0.5 for a in NEUTRAL_ACTIONS},
}

@dataclass
class SynthConfig:
    num_trajectories: int = 4000
    num_actions: int = 7  # 1..7
    seq_len: int = 2      # exactly two actions
    noise_second_action: bool = True
    seed: int = 123

class TwoStepSyntheticDataset(Dataset):
    def __init__(self, cfg: SynthConfig, action_vocab, block_size: int):
        super().__init__()
        self.cfg = cfg
        self.action_vocab = action_vocab
        self.block_size = block_size
        self.rng = np.random.default_rng(cfg.seed)

        first_actions = self.rng.integers(1, cfg.num_actions + 1, size=cfg.num_trajectories)
        if cfg.noise_second_action:
            second_actions = self.rng.integers(1, cfg.num_actions + 1, size=cfg.num_trajectories)
        else:
            second_actions = np.ones(cfg.num_trajectories, dtype=np.int64)

        self.histories = [(int(a1), int(a2)) for a1, a2 in zip(first_actions, second_actions)]

        # Sample winners based on first action's win prob
        winners = []  # 1 or 2
        for a1 in first_actions:
            p = P_WIN[int(a1)]
            winners.append(1 if self.rng.random() < p else 2)
        self.winners = np.array(winners, dtype=np.int64)

    def __len__(self):
        return len(self.histories)

    def __getitem__(self, idx):
        a1, a2 = self.histories[idx]
        history = [a1, a2]

        # Encode actions (no START token in sequence; we follow existing pipeline logic)
        encoded = np.array(action_vocab.encode(history), dtype=np.int32)

        # Pad to block_size (right-aligned like evaluator)
        x = np.zeros(self.block_size, dtype=np.int32)
        x[-len(encoded):] = encoded

        # Value target: one-hot [1,0] if P1 win else [0,1], per-step tiled (length=seq_len)
        winner = self.winners[idx]
        val_onehot = np.array([1, 0], dtype=np.float32) if winner == 1 else np.array([0, 1], dtype=np.float32)
        values = np.tile(val_onehot, (len(encoded), 1))
        # Pad values to block_size
        values_padded = np.zeros((self.block_size, 2), dtype=np.float32)
        values_padded[-len(encoded):] = values

        # Policy target: place mass on the actually-taken action at each step (one-hot over vocab)
        policy = np.zeros((self.block_size, action_vocab.vocab_size), dtype=np.float32)
        # Convert taken actions to vocab indices and mark as 1.0
        for t, act in enumerate(history, start=self.block_size - len(encoded) + 0):
            vocab_idx = action_vocab.stoi[act]
            policy[t, vocab_idx] = 1.0

        actions_t = torch.from_numpy(x)
        policies_t = torch.from_numpy(policy)
        values_t = torch.from_numpy(values_padded)
        return actions_t, policies_t, values_t

# 2) Build loaders (train/val) on the synthetic dataset
synth_cfg = SynthConfig(num_trajectories=6000)
full_ds = TwoStepSyntheticDataset(synth_cfg, action_vocab=action_vocab, block_size=block_size)
# Split 80/20
n_train = int(0.8 * len(full_ds))
train_ds, val_ds = torch.utils.data.random_split(full_ds, [n_train, len(full_ds) - n_train], generator=torch.Generator().manual_seed(42))

train_loader = DataLoader(train_ds, batch_size=256, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=256, shuffle=False)

# 3) Small model for quick convergence (reuse current MODEL_SIZE config if you like)
from rgi.rgizero.models.action_history_transformer import ActionHistoryTransformer
from rgi.rgizero.models.transformer import TransformerConfig

tiny_cfg = TransformerConfig(n_max_context=n_max_context, n_layer=2, n_head=2, n_embd=32)
synth_model = ActionHistoryTransformer(config=tiny_cfg, action_vocab_size=action_vocab.vocab_size, num_players=2).to(device)

# 4) Minimal value-centric training loop (compute our own CE losses to avoid any earlier bugs)
optim = torch.optim.AdamW(synth_model.parameters(), lr=3e-4)
ce = torch.nn.CrossEntropyLoss()

def run_epoch(dl: DataLoader, train: bool) -> tuple[float, float]:
    if train:
        synth_model.train()
    else:
        synth_model.eval()
    total_loss = 0.0
    total_correct = 0
    total = 0
    for actions, policy_t, value_t in dl:
        actions = actions.to(device)
        value_t = value_t.to(device)
        # Winner class index at final step per sample (use last non-zero position)
        # We constructed values as padded with zeros then real at the tail; the final step label is value_t[-1]
        # Compute mask of non-zero rows: but since we fixed length=2, we can take the last row directly
        target = value_t[:, -1, :].argmax(dim=-1)  # (B,) with class 0 or 1

        # Forward (no targets -> get logits for last position)
        with torch.set_grad_enabled(train):
            (policy_logits, value_logits), _ = synth_model(actions)  # value_logits: (B, 2)
            loss = ce(value_logits, target)
            if train:
                optim.zero_grad(set_to_none=True)
                loss.backward()
                torch.nn.utils.clip_grad_norm_(synth_model.parameters(), 1.0)
                optim.step()

        total_loss += loss.item() * actions.size(0)
        preds = value_logits.argmax(dim=-1)
        total_correct += (preds == target).sum().item()
        total += actions.size(0)
    return total_loss / total, total_correct / total

# 5) Train a few epochs and report
for epoch in range(1, 11):
    tr_loss, tr_acc = run_epoch(train_loader, True)
    va_loss, va_acc = run_epoch(val_loader, False)
    print(f"epoch {epoch:02d} | train loss {tr_loss:.4f}, acc {tr_acc*100:5.1f}% | val loss {va_loss:.4f}, acc {va_acc*100:5.1f}%")

# 6) Inspect calibration by first action on validation set
from collections import defaultdict
synth_model.eval()
by_a1 = defaultdict(list)
with torch.no_grad():
    for actions, _, value_t in val_loader:
        actions = actions.to(device)
        (policy_logits, value_logits), _ = synth_model(actions)
        probs_p1 = torch.softmax(value_logits, dim=-1)[:, 0].cpu().numpy()
        targets = value_t[:, -1, 0].cpu().numpy()  # 1 if P1 win else 0
        # Recover first action (unpad): it is the second-to-last token in our construction
        # Find the position of the first non-zero from the left: since we right-pad, first action sits at -2
        first_tokens = actions[:, -2].cpu().numpy()
        first_actions = [action_vocab.decode([int(tok)])[0] for tok in first_tokens]
        for a1, p, y in zip(first_actions, probs_p1, targets):
            by_a1[a1].append((p, y))

print("\nValidation calibration by first action (avg predicted P1 vs empirical win rate):")
for a in sorted(by_a1.keys()):
    arr = np.array(by_a1[a])
    pred = arr[:, 0].mean()
    emp = arr[:, 1].mean()
    n = len(arr)
    print(f"  a1={a}: pred_p1={pred:6.3f}, emp_p1={emp:6.3f}, n={n}")


NameError: name 'action_vocab' is not defined